## Laboratoire 2

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
from torch.autograd import Variable
from deeplib.visualization import make_vizualization_autograd

### Graphe computationnel

In [ ]:
x = Variable(torch.Tensor(3, 3).uniform_(-1, 1), requires_grad=True)
y = Variable(torch.Tensor(3, 3).uniform_(-1, 1), requires_grad=True)
z = Variable(torch.Tensor(3, 3).uniform_(-1, 1), requires_grad=True)
w = torch.matmul(x, y) + x + y + z

In [ ]:
make_vizualization_autograd(w)

### Backprop

### Fonction d'activation